In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# FOLDERNAME = 'CS231n_project/'
# assert FOLDERNAME is not None, "[!] Enter the foldername."

# import sys
# sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# %cd /content/drive/My\ Drive/$FOLDERNAME

# %load_ext autoreload
# %autoreload 2

In [2]:
# !pip install torch==1.7 torchvision==0.8

In [3]:
# %cd approx/src/pytorch/cpp
# !python setup.py install
# %cd ../../../..

In [4]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from conv_norm import PreConv

import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F

import numpy as np
from timeit import default_timer as timer
from utils import ImportanceSampler

USE_GPU = True
dtype = torch.float32 # We will be using float throughout this tutorial.

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [5]:
from utils import get_accuracy, load_dataset
from models import get_model
check_accuracy = lambda loader, model: get_accuracy(loader, model, device, dtype)

In [6]:
def train_model(model_name, dataset_name, model_params={}, hyperparams={}):

  learning_rate = hyperparams.get('lr', 1e-3)
  num_epochs = hyperparams.get('num_epochs', 10)
  weight_decay = hyperparams.get('weight_decay', 0)
  train_ratio = hyperparams.get('train_ratio', 0.8)
  batch_size = hyperparams.get('batch_size', 64)
  seed = hyperparams.get('seed', 0)
  imp_sampling = model_params.get('importance_sampling', False)
  gamma = model_params.get('gamma', 0.9)

  torch.manual_seed(seed)
  np.random.seed(seed)

  loader_train, loader_val, loader_test, num_train, num_channels = load_dataset(dataset_name, train_ratio, batch_size)
  model = get_model(model_name, model_params, learning_rate, loader_train, num_channels, device)

  print("Model architecture:")
  print(model)

  print(f'INFO: Training {model_name} on {dataset_name} with lr {learning_rate}, num_epochs={num_epochs}, weight_decay={weight_decay}')

  optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0)

  epoch_vals = []
  
  weight = torch.tensor([1.0]*num_train)

  t_acc, t_loss = check_accuracy(loader_train, model)
  val_acc, val_loss = check_accuracy(loader_val, model)
  
  start = timer()
  c_time = timer()-start

  print(f'Plot: Train, {0}, {t_loss:.3f}, {t_acc:.2f}, {c_time:.1f}')
  print(f'Plot: Val, {0}, {val_loss:.3f}, {val_acc:.2f}, {c_time:.1f}')

  for e in range(num_epochs):
    model.train()
    doUniform = (e == 0) or (imp_sampling == False)
    loader_train_sampled = loader_train
    if not doUniform:
      train_sampler = ImportanceSampler(num_train, weight, batch_size)
      loader_train_sampled, _, _, _, _ = load_dataset(dataset_name, train_ratio, batch_size, train_sampler)
    
    for t, tpl in enumerate(loader_train_sampled):
        torch.cuda.empty_cache()
        model.train()  # put model to training mode
        x = tpl[0].to(device=device, dtype=dtype)  # move to device, e.g. GPU
        y = tpl[1].to(device=device, dtype=torch.long)

        scores = model(x)
        loss = F.cross_entropy(scores, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if not doUniform:
          idx = tpl[2]
          weight[idx] = gamma * weight[idx] + (1 - gamma) * float(loss)

    t_acc, t_loss = check_accuracy(loader_train, model)
    model.eval()
    val_acc, val_loss = check_accuracy(loader_val, model)
    c_time = timer()-start

    print(f'Plot: Train, {e+1}, {t_loss:.3f}, {t_acc:.2f}, {c_time:.1f}')
    print(f'Plot: Val, {e+1}, {val_loss:.3f}, {val_acc:.2f}, {c_time:.1f}')

  test_acc, test_loss = check_accuracy(loader_test, model)
  print(f'Plot: Test, {val_loss:.3f}, {val_acc:.2f}, {c_time:.1f}')

  return model

In [8]:
# gradinit_params = {
#     "gradinit_iters": 50,
#     "gradinit_alg": "adam", #sgd
#     "gradinit_lr": 1e-2,
#     "gradinit_grad_clip": 100,
# }
model_params = {
#     "gradinit": gradinit_params,
    "convnorm" : {"mode_conv": [('first_frac', 0.25)], "mode_bn": [('first_frac', 0.25)]},
    # "approx_mult" : 0.2,
    # "importance_sampling" : True,
    # "gamma" : 0.9
}
hyperparams = {
    "lr" : 3e-3,
    "num_epochs" : 15,
    "weight_decay" : 0,
    "train_ratio" : 0.8,
    "batch_size" : 1024,
}

def test_setup():
    modeList = [[('first_num', 1)], [('first_frac', 0.25)], [('first_frac', 0.5)], [('first_frac', 0.75)], [('first_frac', 1.0)]][::-1]
    for mode in modeList:
        model_params['convnorm']["mode_conv"] = mode
        model_params['convnorm']["mode_bn"] = mode
        train_model('Resnet18', 'CIFAR100', model_params, hyperparams)

test_setup()

0it [00:00, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
CIFAR100 Train dataset raw mean: 0.4783550798892975, raw std dev: 0.2678655982017517
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
INFO: Size of dataset: Training 40000, Validation 10000, Test 10000
bn_pres is True
Originally num layers were 20
Replace layers list is: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Number of og layers are 20, number of new layers are 20
bn_pres is True
Originally num layers were 20
Replace layers list is: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Number of og layers are 0, number of new layers are 20
Model architecture:
ResNet(
  (conv1): PreConv(
    3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3)
    (bpconv): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=64)
  )
  (bn1): Identity()
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=

/home/ubuntu/CS231n-Faster-Classifier/src/conv_norm.py:112: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/aten/src/ATen/native/SpectralOps.cpp:590.)
  f_input = torch.rfft(cout, 2, normalized=False, onesided=True)
/home/ubuntu/CS231n-Faster-Classifier/src/conv_norm.py:123: UserWarning: The function torch.irfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.ifft or torch.fft.irfft. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/aten/src/ATen/native/SpectralOps.cpp:602.)
  output = torch.irfft(compl_mul_2D(f_input, V), 2, normalized=False, signal_sizes=(final_size_x,final_size_y))


Plot: Train, 0, 6.898, 0.00, 0.0
Plot: Val, 0, 6.918, 0.00, 0.0
Plot: Train, 1, 4.589, 1.04, 74.1
Plot: Val, 1, 4.610, 1.05, 74.1
Plot: Train, 2, 4.468, 3.62, 179.3
Plot: Val, 2, 4.342, 3.47, 179.3
Plot: Train, 3, 4.017, 5.42, 284.5
Plot: Val, 3, 4.180, 5.33, 284.5
Plot: Train, 4, 3.875, 8.42, 391.2
Plot: Val, 4, 3.965, 7.86, 391.2
Plot: Train, 5, 3.428, 12.73, 497.5
Plot: Val, 5, 3.706, 12.03, 497.5
Plot: Train, 6, 3.361, 14.15, 603.4
Plot: Val, 6, 3.693, 12.81, 603.4
Plot: Train, 7, 3.614, 17.41, 710.5
Plot: Val, 7, 3.533, 15.68, 710.5
Plot: Train, 8, 3.004, 21.00, 817.0
Plot: Val, 8, 3.287, 18.45, 817.0
Plot: Train, 9, 3.054, 21.97, 922.9
Plot: Val, 9, 3.354, 18.89, 922.9
Plot: Train, 10, 2.967, 24.27, 1022.4
Plot: Val, 10, 3.199, 20.76, 1022.4
Plot: Train, 11, 2.679, 25.91, 1122.8
Plot: Val, 11, 3.171, 22.04, 1122.8
Plot: Train, 12, 2.732, 29.97, 1225.6
Plot: Val, 12, 3.019, 24.72, 1225.6
Plot: Train, 13, 2.606, 31.81, 1326.6
Plot: Val, 13, 2.986, 25.74, 1326.6
Plot: Train, 14, 2.4

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
INFO: Size of dataset: Training 40000, Validation 10000, Test 10000
bn_pres is True
Originally num layers were 20
Replace layers list is: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Number of og layers are 20, number of new layers are 10
bn_pres is True
Originally num layers were 20
Replace layers list is: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Number of og layers are 10, number of new layers are 10
Model architecture:
ResNet(
  (conv1): PreConv(
    3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3)
    (bpconv): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=64)
  )
  (bn1): Identity()
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): PreConv(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
        (bpconv): Conv2d(64,

Plot: Train, 0, 6.895, 0.00, 0.0
Plot: Val, 0, 6.904, 0.00, 0.0
Plot: Train, 1, 3.883, 16.52, 59.8
Plot: Val, 1, 3.649, 15.19, 59.8
Plot: Train, 2, 2.905, 26.05, 119.0
Plot: Val, 2, 3.087, 22.91, 119.0
Plot: Train, 3, 2.751, 32.28, 178.2
Plot: Val, 3, 3.107, 26.13, 178.2
Plot: Train, 4, 2.340, 36.18, 237.2
Plot: Val, 4, 2.859, 28.93, 237.2
Plot: Train, 5, 2.365, 39.09, 297.1
Plot: Val, 5, 2.825, 29.82, 297.1
Plot: Train, 6, 1.653, 45.77, 356.2
Plot: Val, 6, 2.768, 32.71, 356.2
Plot: Train, 7, 1.837, 55.81, 415.7
Plot: Val, 7, 2.715, 36.28, 415.7
Plot: Train, 8, 1.508, 55.18, 474.6
Plot: Val, 8, 2.848, 34.26, 474.6
Plot: Train, 9, 1.253, 61.87, 534.3
Plot: Val, 9, 2.953, 35.30, 534.3
Plot: Train, 10, 0.867, 70.69, 603.9
Plot: Val, 10, 3.082, 36.13, 603.9
Plot: Train, 11, 0.813, 75.88, 715.2
Plot: Val, 11, 2.851, 35.87, 715.2
Plot: Train, 12, 0.813, 80.92, 826.3
Plot: Val, 12, 3.105, 36.95, 826.3
Plot: Train, 13, 0.579, 79.03, 936.5
Plot: Val, 13, 3.327, 35.61, 936.5
Plot: Train, 15, 0.1

In [9]:
1

1